In [2]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Cleaning

In [4]:
data = pd.read_csv('Building_Energy_Benchmarking.csv')
data.head()

<ipython-input-4-68005734ec66>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Building_Energy_Benchmarking.csv')


,X,Y,PID,SSL,PMPROPERTYID,PROPERTYNAME,PMPARENTPROPERTYID,PARENTPROPERTYNAME,REPORTINGYEAR,REPORTSTATUS,ADDRESSOFRECORD,OWNEROFRECORD,WARD,REPORTEDADDRESS,CITY,STATE,POSTALCODE,YEARBUILT,PRIMARYPROPERTYTYPE_SELFSELECT,PRIMARYPROPERTYTYPE_EPACALC,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,ENERGYSTARSCORE,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERSCORE_MFPROPERTIES,WATERUSE_ALLWATERSOURCES_KGAL,METEREDAREAS_ENERGY,METEREDAREAS_WATER,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,GIS_LAST_MOD_DTTM,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER,UBID,LASTUPDATE,OBJECTID
0,-8.575303e+06,4.707303e+06,DC02220020,0222 0020,NaN,NaN,NaN,NaN,2021,Exempt from this year's disclosure,727 15TH ST NW,NATIONAL COMMUNITY REINVESTMENT COALITION,2.0,NaN,WASHINGTON,DC,20005,1985.0,NaN,NaN,46332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899610,-77.033254,218739.0,397115.369981,136862.520445,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/09/12 04:00:00+00,3541768
1,-8.575227e+06,4.707286e+06,DC02220027,0222 0027,NaN,NaN,NaN,NaN,2021,Exempt from this year's disclosure,1425 NEW YORK AVE NW,NY AVENUE TITLEHOLDER LLC,2.0,NaN,WASHINGTON,DC,20005,1991.0,NaN,NaN,158124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899493,-77.032576,300903.0,397174.190168,136849.566803,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/04/14 04:00:00+00,3541769
2,-8.575234e+06,4.707311e+06,DC02220027,0222 0027,NaN,NaN,NaN,NaN,2022,Exempt from this year's disclosure,1425 NEW YORK AVENUE NW,NY AVENUE TITLEHOLDER LLC,2.0,NaN,WASHINGTON,DC,20005,1991.0,NaN,NaN,158124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899669,-77.032635,300903.0,397169.119971,136869.030539,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/04/14 04:00:00+00,3541770
3,-8.575281e+06,4.707161e+06,DC02230025,0223 0025,NaN,NaN,NaN,NaN,2012,No Report Received,1435 G ST NW,1440 NEW YORK AVENUE CORPORATION,2.0,NaN,WASHINGTON,DC,NaN,1934.0,NaN,NaN,228254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.898618,-77.033060,814459.0,397132.162500,136752.413602,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3541771
4,-8.575222e+06,4.707022e+06,DC02240824,0224 0824,NaN,NaN,NaN,NaN,2013,Exempt from this year's disclosure,1401 F ST NW,FSP HAMILTON SQUARE LLC,2.0,NaN,WASHINGTON,DC,20005,1999.0,NaN,NaN,269472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.897644,-77.032531,217997.0,397178.028700,136644.297802,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/06/06 04:00:00+00,3541772


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26577 entries, 0 to 26576
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   X                               26577 non-null  float64
 1   Y                               26577 non-null  float64
 2   PID                             26577 non-null  object 
 3   SSL                             26577 non-null  object 
 4   PMPROPERTYID                    22224 non-null  float64
 5   PROPERTYNAME                    22142 non-null  object 
 6   PMPARENTPROPERTYID              937 non-null    object 
 7   PARENTPROPERTYNAME              937 non-null    object 
 8   REPORTINGYEAR                   26577 non-null  int64  
 9   REPORTSTATUS                    26577 non-null  object 
 10  ADDRESSOFRECORD                 26573 non-null  object 
 11  OWNEROFRECORD                   26571 non-null  object 
 12  WARD                            

In [6]:
data['PRIMARYPROPERTYTYPE_SELFSELECT'].unique()

array([nan, 'Office', 'Self-Storage Facility', 'Hotel',
       'Multifamily Housing', 'K-12 School', 'Residence Hall/Dormitory',
       'Medical Office', 'Mixed Use Property', 'Strip Mall',
       'Retail Store', 'Manufacturing/Industrial Plant',
       'Residential Care Facility', 'Other',
       'Hospital (General Medical & Surgical)',
       'Fitness Center/Health Club/Gym', 'Stadium (Open)',
       'Police Station', 'Other - Lodging/Residential',
       'Non-Refrigerated Warehouse', 'Other - Public Services',
       'Swimming Pool', 'Library', 'Pre-school/Daycare',
       'Repair Services (Vehicle, Shoe, Locksmith, etc)',
       'Social/Meeting Hall', 'Prison/Incarceration', 'Laboratory',
       'Fire Station', 'Adult Education', 'Food Sales',
       'Vehicle Repair Services', 'Single Family Home',
       'Bank/Financial Institution', 'Supermarket/Grocery Store',
       'College/University', 'Museum', 'Bank Branch', 'Financial Office',
       'Senior Care Community', 'Senior Living

In [7]:
university_property_types = ['College/University (Campus-Level)', 'College/University', 'Residence Hall/Dormitory']
owner_keywords = ['George Washington', 'Howard', 'Georgetown University,' 'American University', 'Catholic University', 'University']
# Create a boolean mask for the OWNEROFRECORD condition
owner_mask = data['OWNEROFRECORD'].str.contains('|'.join(owner_keywords), na=False)

# Filter the DataFrame based on either condition
uni_data = data[data['PRIMARYPROPERTYTYPE_SELFSELECT'].isin(university_property_types) | owner_mask]
with pd.option_context('display.max_rows', None):
    print(uni_data.isna().sum())

X                                   0
Y                                   0
PID                                 0
SSL                                 0
PMPROPERTYID                        0
PROPERTYNAME                        0
PMPARENTPROPERTYID                418
PARENTPROPERTYNAME                418
REPORTINGYEAR                       0
REPORTSTATUS                        0
ADDRESSOFRECORD                     0
OWNEROFRECORD                       0
WARD                                8
REPORTEDADDRESS                     0
CITY                               10
STATE                              10
POSTALCODE                         77
YEARBUILT                           8
PRIMARYPROPERTYTYPE_SELFSELECT      0
PRIMARYPROPERTYTYPE_EPACALC        48
TAXRECORDFLOORAREA                  0
REPORTEDBUILDINGGROSSFLOORAREA      0
ENERGYSTARSCORE                   546
SITEEUI_KBTU_FT                     0
WEATHERNORMALZEDSITEEUI_KBTUFT      0
SOURCEEUI_KBTU_FT                   0
WEATHERNORMA

In [8]:
uni_data.shape

(788, 73)

In [9]:
uni_data = uni_data.drop('UBID', axis=1)

In [10]:
owner_replacements = {
    'THE GEORGE WASHINGTON UNIVERSITY': 'George Washington University',
    'GEORGE WASHINGTON UNIVERSITY': 'George Washington University',
    'HOWARD UNIVERSITY (LAND ONLY)': 'Howard University',
    'HOWARD UNIVERSITY' : 'Howard University',
    'THE HOWARD UNIVERSITY' : 'Howard University',
    'THE HOWARD UNIVERSITY INC' : 'Howard University',
    'HOWARD DORMITORY HOLDINGS 1 LLC' : 'Howard University',
    'THE CATHOLIC UNIVERSITY OF AMERICA' : 'Catholic University of America',
    'GEORGETOWN UNIVERSITY' : 'Georgetown University',
    'GEORGETOWN COLLEGE LAW CENTER' : 'Georgetown University',
    'AMERICAN UNIVERSITY' : 'American University',
    'PRESIDENT & DIRECTORS OF GEORGETOWN UNIVERSITY' : 'Georgetown University',
    'JESUIT COMMUNITY AT GEORGETOWN UNIV INC' : 'Georgetown University',
    'THE PRESIDENT AND DIRECTORS OF GEORGETOWN COLLEGE' : 'Georgetown University',
    'CATHOLIC UNIVERSITY OF AMERICA' : 'Catholic University of America',
    'UNIVERSITY OF THE DISTRICT OF COLUMBIA' : 'University of District of Columbia',
    'TRINITY COLLEGE' : 'Trinity College',
    'GALLAUDET UNIVERSITY' : 'Gallaudet University'
    # 'JOHNS HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    # 'THE JOHN HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    # 'THE JOHNS HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    # 'ASB 801 NORTH CAPITOL LLC' : 'HOWARD UNIVERSITY',
    # 'ASB 601 NORTH CAPITOL LLC' : 'HOWARD UNIVERSITY',
    # 'GREEN HARRIS LLC' : 'GEORGETOWN UNIVERSITY',
    # "JEMAL'S CAYRE CARBARN LLC" : 'GEORGETOWN UNIVERSITY',
    # 'THE REGENTS OF THE UNIVERSITY OF CALIFORNIA' : 'UNIVERSITY OF CALIFORNIA WASHINGTON CENTER',
    # 'THEOLOGICAL COLLEGE INC' : 'THEOLOGICAL COLLEGE',
    # 'TRUSTEES FOR HARVARD UNIVERSITY' : 'HARDWARD UNIVERSITY',
    # 'DISTRICT OF COLUMBIA' : 'UNIVERSITY OF THE DISTRICT OF COLUMBIA'
}

def clean_owner(owner):
    if owner in owner_replacements:
        owner = owner_replacements[owner]
    return owner

In [11]:
uni_data['OWNEROFRECORD'] = uni_data['OWNEROFRECORD'].apply(clean_owner)
uni_data = uni_data[uni_data['OWNEROFRECORD'].isin(['George Washington University', 'Howard University', 'Catholic University of America',
                                     'Georgetown University', 'American University', 'University of District of Columbia', 'Trinity College', 'Gallaudet University'])]
uni_data.shape

(618, 72)

In [12]:
def clean_address(address):
  address = address.upper()
  address = re.sub(r'\bST\b', 'STREET', address)
  address = re.sub(r'\bAVE\b', 'AVENUE', address)
  address = re.sub(r'\bAV\b', 'AVENUE', address)
  address = re.sub(r'\bRD\b', 'ROAD', address)
  address = re.sub(r'\bBLVD\b', 'BOULEVARD', address)
  address = address.lstrip('0')
  # Adding space between street name and directional indicators if missing
  address = re.sub(r'(\w)(NW|NE|SW|SE)', r'\1 \2', address)

  address = address.replace("MASSACHU SETTS", "MASSACHUSETTS")
  address = address.replace("CON NECTICUT", "CONNECTICUT")
  address = address.replace("RE SERVOIR", "RESERVOIR")
  return address

def add_missing_quadrants(df, column):
    # Compile regex for quadrant detection
    quadrant_pattern = re.compile(r'\b(NE|NW|SE|SW)\b$')

    # Create a dictionary to map base addresses to their quadrants
    base_address_map = {}
    for address in df[column].unique():
        base_address = re.sub(r'\b(NE|NW|SE|SW)\b$', '', address).strip()
        quadrant = quadrant_pattern.search(address)
        if quadrant:
            base_address_map[base_address] = quadrant.group()

    # Function to append missing quadrants
    def append_quadrant(address):
        base_address = re.sub(r'\b(NE|NW|SE|SW)\b$', '', address).strip()
        if not quadrant_pattern.search(address) and base_address in base_address_map:
            return f"{address} {base_address_map[base_address]}"
        return address

    df[column] = df[column].apply(append_quadrant)
    return df

In [13]:
uni_data['ADDRESSOFRECORD'] = uni_data['ADDRESSOFRECORD'].apply(clean_address)
uni_data = add_missing_quadrants(uni_data, 'ADDRESSOFRECORD')

In [14]:
print(uni_data['ADDRESSOFRECORD'].nunique())
uni_data['ADDRESSOFRECORD'].unique()

84


array(['2350 H STREET NW', '2100 FOXHALL ROAD NW', '607 23RD STREET NW',
       '2021 F STREET NW', '4TH STREET NE', '125 MICHIGAN AVENUE NE',
       '800 21ST STREET NW', '2212 I STREET NW', '2100 F STREET NW',
       '2201 VIRGINIA AVENUE NW', '616 23RD STREET NW',
       '2223 H STREET NW', '1900 F STREET NW', '514 19TH STREET NW',
       '736 22ND STREET NW', '2115 F STREET NW', '2110 G STREET NW',
       '600 20TH STREET NW', '2601 VIRGINIA AVENUE NW',
       '1129 NEW HAMPSHIRE AVENUE NW', '2109 F STREET NW',
       '120 F STREET NW', '101 F STREET NW', '111 G STREET NW',
       '550 1ST STREET NW', '4400 MASSACHUSETTS AVENUE NW',
       '2013 H STREET NW', '805 21ST STREET NW', '600 22ND STREET NW',
       '1922 F STREET NW', '2001 - 2033 G STREET NW', '2033 G STREET NW',
       '714 21ST STREET NW', '2135 F STREET NW', '2201 G STREET NW',
       '801 22ND STREET NW', '2300 I STREET NW',
       '4801 MASSACHUSETTS AVENUE NW', '950 NEW HAMPSHIRE AVENUE',
       '3515 PROSPECT STR

# EDA

In [15]:
fig = px.scatter_mapbox(uni_data, lat="LATITUDE", lon="LONGITUDE", color="OWNEROFRECORD",
                        hover_name="PROPERTYNAME", hover_data=["ADDRESSOFRECORD"],
                        title="Map of University Buildings, Washington, DC")
fig.update_layout(mapbox_style="carto-positron")
fig.show()

## Comparison of Universities Based on Cumulative Energy Usage



In [16]:
def plot_university_efficiency_aggregated(df, feature, year=None):
    """
    Plots a bar chart comparing the selected feature usage across universities for the given year.
    If the year is not specified, it uses the most recent available record for each building.
    """
    if year is not None:
      # Filter data for the specified year
      df_year = df[df['REPORTINGYEAR'] == year]
    else:
      # If year is not specified, get the last available record for each building of each university
      df = df.sort_values(by=['OWNEROFRECORD', 'PROPERTYNAME', 'REPORTINGYEAR'])  # Sort by university, building, and year
      df_year = df.groupby(['OWNEROFRECORD', 'PROPERTYNAME']).tail(1)  # Select last record per university-building pairing

    # Aggregate the total feature usage per university
    university_totals = df_year.groupby('OWNEROFRECORD')[feature].sum().reset_index()

    # Create the bar plot
    bar_plot = px.bar(
        university_totals,
        x='OWNEROFRECORD',
        y=feature,
        title=f"University Comparison of {feature} ({year if year else 'Most Recent Record'})",
        labels={feature: f"{feature} (Total)", 'OWNEROFRECORD': 'University'},
        color='OWNEROFRECORD',  # Assign each university a unique color
        color_continuous_scale='Viridis'
    )

    # Update layout for better readability and aesthetics
    bar_plot.update_layout(
        # xaxis_title="University",
        # yaxis_title=f"{feature} (Total)",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12, color="black"),
        xaxis_tickangle=-45  # Rotate x-axis labels for readability
    )

    # Customize tooltip
    bar_plot.update_traces(
        hovertemplate='University=%{x}<br>' + f'{feature}=%{{y}}'
    )

    # Add a legend and adjust layout to match aesthetic
    bar_plot.update_layout(
        showlegend=True,
        legend_title_text="University",
        margin=dict(t=60, b=120)
    )

    return bar_plot

In [17]:
features = ['SOURCEEUI_KBTU_FT', 'TOTGHGEMISSIONS_METRICTONSCO2E', 'WATERUSE_ALLWATERSOURCES_KGAL']
for feature in features:
  fig = plot_university_efficiency_aggregated(uni_data, feature)
  fig.show()

c

## Comparison of Universities Based on Average Energy Usage

In [18]:
def plot_university_efficiency_average(df, feature, year=None):
    """
    Plots a bar chart comparing the average feature usage across universities for the given year.
    If the year is not specified, it uses the most recent available record for each building.
    """
    if year is not None:
        # Filter data for the specified year
        df_year = df[df['REPORTINGYEAR'] == year]
    else:
        # If year is not specified, get the last available record for each building of each university
        df = df.sort_values(by=['OWNEROFRECORD', 'PROPERTYNAME', 'REPORTINGYEAR'])  # Sort by university, building, and year
        df_year = df.groupby(['OWNEROFRECORD', 'PROPERTYNAME']).tail(1)  # Select last record per university-building pairing

    # Calculate the average feature usage per university
    university_avg = df_year.groupby('OWNEROFRECORD').agg(
        avg_feature=(feature, 'mean'),  # Calculate the average feature per building
        building_count=('PROPERTYNAME', 'nunique')  # Count unique buildings per university
    ).reset_index()

    # Create the bar plot
    bar_plot = px.bar(
        university_avg,
        x='OWNEROFRECORD',
        y='avg_feature',
        title=f"University Comparison of Average {feature} ({year if year else 'Most Recent Record'})",
        labels={'avg_feature': f"Average {feature}", 'OWNEROFRECORD': 'University'},
        color='OWNEROFRECORD',  # Assign each university a unique color
        color_continuous_scale='Viridis'
    )

    # Update layout for better readability and aesthetics
    bar_plot.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12, color="black"),
        xaxis_tickangle=-45  # Rotate x-axis labels for readability
    )

    # Customize tooltip to show university, average feature value, and building count
    bar_plot.update_traces(
        hovertemplate=(
            'University=%{x}<br>' +
            f'Average {feature}=%{{y}}<br>' +
            'Building Count=%{customdata[0]}'
        ),
        customdata=university_avg[['building_count']]  # Pass building count data for tooltip
    )

    # Add a legend and adjust layout to match aesthetic
    bar_plot.update_layout(
        showlegend=True,
        legend_title_text="University",
        margin=dict(t=60, b=120)
    )

    return bar_plot

In [19]:
features = ['SOURCEEUI_KBTU_FT', 'TOTGHGEMISSIONS_METRICTONSCO2E', 'WATERUSE_ALLWATERSOURCES_KGAL']
for feature in features:
  fig = plot_university_efficiency_average(uni_data, feature)
  fig.show()

## Time Series Analysis of Universities Based on Cumulative Energy Usage

In [20]:
def plot_university_time_series(df, feature):
    """
    Plots a time series chart comparing energy usage for a specific feature across universities over the years.
    Each line in the plot represents a university's feature usage over time.
    """

    # Ensure the feature is numeric
    df[feature] = pd.to_numeric(df[feature], errors='coerce')

    # Drop rows where the feature is NaN
    df = df.dropna(subset=[feature])
    # Aggregate the total feature usage per university and year
    university_yearly_totals = df.groupby(['OWNEROFRECORD', 'REPORTINGYEAR'])[feature].sum().reset_index()

    # Create the line plot
    time_series_plot = px.line(
        university_yearly_totals,
        x='REPORTINGYEAR',  # Time axis (Year)
        y=feature,  # Feature to plot
        color='OWNEROFRECORD',  # Different lines for each university
        title=f"University Comparison of {feature} Over the Years",
        labels={'REPORTINGYEAR': 'Year', feature: f"{feature} (Total)", 'OWNEROFRECORD': 'University'},
        markers=True  # Adds markers at each point on the line for better visibility
    )

    # Update layout for better readability
    time_series_plot.update_layout(
        xaxis_title="Year",
        yaxis_title=f"{feature} (Total)",
        plot_bgcolor='white',  # Clean white background for a professional look
        paper_bgcolor='white',  # White background for the entire plot
        font=dict(family="Arial", size=12, color="black"),
        showlegend=True  # Show legend for interactivity
    )

    # Customize tooltip
    time_series_plot.update_traces(
        hovertemplate='University=%{x}<br>' + f'{feature}=%{{y}}'
    )

    return time_series_plot


In [21]:
for feature in features:
  fig = plot_university_time_series(uni_data, feature)
  fig.show()

## Time Series Analysis of Universities Based on Average Energy Usage

In [22]:
def plot_university_time_series_avg(df, feature):
    """
    Plots a time series chart comparing the average energy usage for a specific feature across universities over the years.
    Each line in the plot represents a university's average feature usage per building over time.
    """
    # Ensure the feature is numeric
    df[feature] = pd.to_numeric(df[feature], errors='coerce')

    # Drop rows where the feature is NaN
    df = df.dropna(subset=[feature])

    # Calculate total feature usage and count of buildings per university per year
    university_yearly_totals = (
        df.groupby(['OWNEROFRECORD', 'REPORTINGYEAR'])[feature]
        .agg(['sum', 'count'])
        .reset_index()
        .rename(columns={'sum': 'total_usage', 'count': 'building_count'})
    )

    # Calculate average usage per building
    university_yearly_totals['average_usage'] = (
        university_yearly_totals['total_usage'] / university_yearly_totals['building_count']
    )

    # Create the line plot
    time_series_plot = px.line(
        university_yearly_totals,
        x='REPORTINGYEAR',  # Time axis (Year)
        y='average_usage',  # Average usage to plot
        color='OWNEROFRECORD',  # Different lines for each university
        title=f"University Comparison of Average {feature} Usage Per Building Over the Years",
        labels={'REPORTINGYEAR': 'Year', 'average_usage': f"Average {feature} Usage", 'OWNEROFRECORD': 'University'},
        markers=True  # Adds markers at each point on the line for better visibility
    )

    # Update layout for better readability
    time_series_plot.update_layout(
        xaxis_title="Year",
        yaxis_title=f"Average {feature} Usage",
        plot_bgcolor='white',  # Clean white background for a professional look
        paper_bgcolor='white',  # White background for the entire plot
        font=dict(family="Arial", size=12, color="black"),
        showlegend=True  # Show legend for interactivity
    )

    # Customize tooltip
    time_series_plot.update_traces(
        hovertemplate='University=%{x}<br>' + f'Average {feature}=%{{y}}'
    )

    return time_series_plot

In [23]:
for feature in features:
  fig = plot_university_time_series_avg(uni_data, feature)
  fig.show()

## Analysis on University Buildings

In [24]:
def plot_building_efficiency_by_university(df, feature, university_name, year=None):
    """
    Plots a bar chart comparing the selected feature usage across buildings for a specified university and year.
    If the year is not specified, it uses the most recent available record for each building.
    """
    # Filter data for the selected university
    df_university = df[df['OWNEROFRECORD'] == university_name]

    if year is not None:
        # Further filter data for the specified year
        df_year = df_university[df_university['REPORTINGYEAR'] == year]
    else:
        # If year is not specified, get the last available record for each building
        df_university = df_university.sort_values(by=['PROPERTYNAME', 'REPORTINGYEAR'])  # Sort by building and year
        df_year = df_university.groupby('PROPERTYNAME').tail(1)  # Select last record per building

    # Aggregate the total feature usage per building and sort in descending order
    building_totals = df_year.groupby('PROPERTYNAME')[feature].sum().reset_index()
    building_totals = building_totals.sort_values(by=feature, ascending=False)  # Sort by feature in descending order

    bar_plot = px.bar(
        building_totals,
        x='PROPERTYNAME',
        y=feature,
        title=f"{university_name} Building Comparison of {feature} ({year if year else 'Most Recent Record'})",
        labels={feature: f"{feature} (Total)", 'PROPERTYNAME': 'Building'},
        text_auto=True,
        color='PROPERTYNAME',  # Assign each building a unique color
        color_continuous_scale='Viridis'
    )

    bar_plot.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12, color="black"),
        xaxis_tickangle=-45  # Rotate x-axis labels for readability
    )

    # Customize tooltip to show building and feature value
    bar_plot.update_traces(
        hovertemplate='Building=%{x}<br>' + f'{feature}=%{{y}}'
    )

    # Add a legend and adjust layout to match aesthetic
    bar_plot.update_layout(
        showlegend=True,
        legend_title_text="Building",
        margin=dict(t=60, b=120)
    )

    return bar_plot


In [25]:
for feature in features:
  fig = plot_building_efficiency_by_university(uni_data, feature, "George Washington University")
  fig.show()

## Carto-Positron Plot of University Buildings

In [26]:
def plot_eui_buildings_on_map(df, university_name=None, year=None, threshold=180.6):
    # Filter data based on the university name if provided
    if university_name is not None:
        df_university = df[df['OWNEROFRECORD'] == university_name].copy()
    else:
        df_university = df.copy()  # Use all records if no university name is given

    # Determine if we are using a specified year or the most recent available record per building
    if year is not None:
        # Filter data for the specified year
        df_year = df_university[df_university['REPORTINGYEAR'] == year]
    else:
        # If year is not specified, get the last available record for each building
        df_university = df_university.sort_values(by=['OWNEROFRECORD', 'PROPERTYNAME', 'REPORTINGYEAR'])
        df_year = df_university.groupby(['OWNEROFRECORD', 'PROPERTYNAME']).tail(1)  # Select last record per university-building pairing

    # Check if necessary columns are present
    if 'LONGITUDE' not in df.columns or 'LATITUDE' not in df.columns or 'SOURCEEUI_KBTU_FT' not in df.columns:
        raise ValueError("The dataset must contain 'Longitude', 'Latitude', and 'SOURCEEUI_KBTU_FT' columns.")

    # Color-code buildings based on the threshold value of SOURCEEUI_KBTU_FT
    df_year['Color'] = df_year['SOURCEEUI_KBTU_FT'].apply(lambda x: "Source EUI above 180.6" if x > threshold else "Source EUI below or equal to 180.6")

    # Create the map plot
    fig = px.scatter_mapbox(
        df_year,
        lat="LATITUDE",
        lon="LONGITUDE",
        color="Color",
        hover_name="PROPERTYNAME",
        hover_data={"SOURCEEUI_KBTU_FT": True, "OWNEROFRECORD": True, "Color": False},
        title=f"{university_name if university_name else 'All Universities'} Buildings in {year if year else 'Most Recent Year Available'} (Red: > {threshold} SOURCEEUI_KBTU_FT, Green: ≤ {threshold})",
        zoom=13,
        height=600,
        color_discrete_map={"Source EUI above 180.6": "red", "Source EUI below or equal to 180.6": "green"}  # Define the colors for the markers
    )

    # Update map layout for Carto Positron style
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_center={"lat": df_year["LATITUDE"].mean(), "lon": df_year["LONGITUDE"].mean()},
        margin={"r": 0, "t": 50, "l": 0, "b": 0}
    )

    return fig

In [27]:
fig = plot_eui_buildings_on_map(uni_data, "George Washington University")
fig.show()

<ipython-input-26-cfc0c52f5268>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
plot_eui_buildings_on_map(uni_data)

<ipython-input-26-cfc0c52f5268>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
def plot_eui_buildings_on_map(df, year=None, threshold=180.6):
    # Standardize university names for consistency
    df['OWNEROFRECORD'] = df['OWNEROFRECORD'].str.strip().str.title()

    #print("Unique university names in OWNEROFRECORD column:", df['OWNEROFRECORD'].unique())

    fig = go.Figure()

    # Determine if we are using a specified year or the most recent available record per building
    if year is not None:
        df = df[df['REPORTINGYEAR'] == year]
    else:
        # Get the most recent record for each building if no year is specified
        df = df.sort_values(by=['OWNEROFRECORD', 'PROPERTYNAME', 'REPORTINGYEAR'])
        df = df.groupby(['OWNEROFRECORD', 'PROPERTYNAME']).tail(1)

    # Check if necessary columns are present
    if 'LONGITUDE' not in df.columns or 'LATITUDE' not in df.columns or 'SOURCEEUI_KBTU_FT' not in df.columns:
        raise ValueError("The dataset must contain 'LONGITUDE', 'LATITUDE', and 'SOURCEEUI_KBTU_FT' columns.")

    # Iterate through each university
    for university in df['OWNEROFRECORD'].unique():
        df_university = df[df['OWNEROFRECORD'] == university]

        # Separate the data based on the threshold
        df_above_threshold = df_university[df_university['SOURCEEUI_KBTU_FT'] > threshold]
        df_below_threshold = df_university[df_university['SOURCEEUI_KBTU_FT'] <= threshold]

        # Plot buildings above the threshold
        fig.add_trace(go.Scattermapbox(
            lat=df_above_threshold['LATITUDE'],
            lon=df_above_threshold['LONGITUDE'],
            mode='markers',
            marker=dict(size=6, color='red'),
            name=f"{university} (Above {threshold})",
            legendgroup=university,
            legendgrouptitle_text=university,  # Ensure each group has a title
            hovertext=df_above_threshold['PROPERTYNAME'],
            hoverinfo='text',
            customdata=df_above_threshold[['SOURCEEUI_KBTU_FT', 'OWNEROFRECORD']],
            hovertemplate="<b>%{hovertext}</b><br>" +
                          "University: %{customdata[1]}<br>" +
                          "EUI: %{customdata[0]:.2f} kBtu/ft²<extra></extra>"
        ))

        # Plot buildings below or equal to the threshold
        fig.add_trace(go.Scattermapbox(
            lat=df_below_threshold['LATITUDE'],
            lon=df_below_threshold['LONGITUDE'],
            mode='markers',
            marker=dict(size=6, color='green'),
            name=f"{university} (Below {threshold})",
            legendgroup=university,
            legendgrouptitle_text=university,  # Re-ensure title for each group
            hovertext=df_below_threshold['PROPERTYNAME'],
            hoverinfo='text',
            customdata=df_below_threshold[['SOURCEEUI_KBTU_FT', 'OWNEROFRECORD']],
            hovertemplate="<b>%{hovertext}</b><br>" +
                          "University: %{customdata[1]}<br>" +
                          "EUI: %{customdata[0]:.2f} kBtu/ft²<extra></extra>"
        ))

    # Update layout for map and legend
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_center={"lat": df["LATITUDE"].mean(), "lon": df["LONGITUDE"].mean()},
        mapbox_zoom=13,
        title=f"All Universities Buildings in {year if year else 'Most Recent Year Available'}",
        height=600,
        margin={"r": 0, "t": 50, "l": 0, "b": 0}
    )

    return fig

In [43]:
plot_eui_buildings_on_map(uni_data)